In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 12)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=4096,
    buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
# model.set_parameters("trained_models/g1-reptile/sac")
# model.batch_size = 2048 * 5

In [ ]:
model.learn(
    total_timesteps=int(1e8), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 174       |
|    ep_rew_mean     | -1.18e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 195       |
|    time_elapsed    | 96        |
|    total_timesteps | 18828     |
| train/             |           |
|    actor_loss      | 16.4      |
|    critic_loss     | 3.02      |
|    ent_coef        | 0.86      |
|    ent_coef_loss   | -2.48     |
|    learning_rate   | 0.0001    |
|    n_updates       | 1560      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 185       |
|    ep_rew_mean     | -1.01e+03 |
| time/              |           |
|    episodes        | 200       |
|    fps             | 192       |
|    time_elapsed    | 192       |
|    total_timesteps | 37080     |
| train/             |           |
|    actor_loss      | 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 89.7     |
|    ep_rew_mean     | -326     |
| time/              |          |
|    episodes        | 1600     |
|    fps             | 190      |
|    time_elapsed    | 1233     |
|    total_timesteps | 234408   |
| train/             |          |
|    actor_loss      | 173      |
|    critic_loss     | 52.8     |
|    ent_coef        | 0.156    |
|    ent_coef_loss   | -11.4    |
|    learning_rate   | 0.0001   |
|    n_updates       | 19525    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 79.9     |
|    ep_rew_mean     | -284     |
| time/              |          |
|    episodes        | 1700     |
|    fps             | 190      |
|    time_elapsed    | 1275     |
|    total_timesteps | 242448   |
| train/             |          |
|    actor_loss      | 175      |
|    critic_loss     | 61.8     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 62.6     |
|    ep_rew_mean     | -203     |
| time/              |          |
|    episodes        | 3100     |
|    fps             | 189      |
|    time_elapsed    | 1784     |
|    total_timesteps | 338328   |
| train/             |          |
|    actor_loss      | 163      |
|    critic_loss     | 68.2     |
|    ent_coef        | 0.0867   |
|    ent_coef_loss   | -1.22    |
|    learning_rate   | 0.0001   |
|    n_updates       | 28185    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 61.2     |
|    ep_rew_mean     | -196     |
| time/              |          |
|    episodes        | 3200     |
|    fps             | 189      |
|    time_elapsed    | 1816     |
|    total_timesteps | 344292   |
| train/             |          |
|    actor_loss      | 165      |
|    critic_loss     | 71.2     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 47.7     |
|    ep_rew_mean     | -159     |
| time/              |          |
|    episodes        | 4600     |
|    fps             | 189      |
|    time_elapsed    | 2227     |
|    total_timesteps | 421680   |
| train/             |          |
|    actor_loss      | 140      |
|    critic_loss     | 79.5     |
|    ent_coef        | 0.0868   |
|    ent_coef_loss   | 0.316    |
|    learning_rate   | 0.0001   |
|    n_updates       | 35131    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 48.8     |
|    ep_rew_mean     | -168     |
| time/              |          |
|    episodes        | 4700     |
|    fps             | 189      |
|    time_elapsed    | 2252     |
|    total_timesteps | 426456   |
| train/             |          |
|    actor_loss      | 140      |
|    critic_loss     | 66.4     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 41.2     |
|    ep_rew_mean     | -125     |
| time/              |          |
|    episodes        | 6100     |
|    fps             | 189      |
|    time_elapsed    | 2586     |
|    total_timesteps | 489192   |
| train/             |          |
|    actor_loss      | 115      |
|    critic_loss     | 66.7     |
|    ent_coef        | 0.0964   |
|    ent_coef_loss   | 0.223    |
|    learning_rate   | 0.0001   |
|    n_updates       | 40757    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 43.1     |
|    ep_rew_mean     | -172     |
| time/              |          |
|    episodes        | 6200     |
|    fps             | 189      |
|    time_elapsed    | 2607     |
|    total_timesteps | 493176   |
| train/             |          |
|    actor_loss      | 113      |
|    critic_loss     | 67       |
|    ent_coef 

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 70.4     |
|    ep_rew_mean     | 35.1     |
| time/              |          |
|    episodes        | 100600   |
|    fps             | 189      |
|    time_elapsed    | 26419    |
|    total_timesteps | 4996428  |
| train/             |          |
|    actor_loss      | 13.3     |
|    critic_loss     | 0.569    |
|    ent_coef        | 0.0625   |
|    ent_coef_loss   | -0.0534  |
|    learning_rate   | 0.0001   |
|    n_updates       | 416360   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 62.8     |
|    ep_rew_mean     | 22.3     |
| time/              |          |
|    episodes        | 100700   |
|    fps             | 189      |
|    time_elapsed    | 26452    |
|    total_timesteps | 5002632  |
| train/             |          |
|    actor_loss      | 12.8     |
|    critic_loss     | 0.57     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 66.4     |
|    ep_rew_mean     | 27       |
| time/              |          |
|    episodes        | 102100   |
|    fps             | 189      |
|    time_elapsed    | 26950    |
|    total_timesteps | 5096832  |
| train/             |          |
|    actor_loss      | 12.2     |
|    critic_loss     | 0.619    |
|    ent_coef        | 0.0625   |
|    ent_coef_loss   | -0.208   |
|    learning_rate   | 0.0001   |
|    n_updates       | 424727   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 70.1     |
|    ep_rew_mean     | 36.9     |
| time/              |          |
|    episodes        | 102200   |
|    fps             | 189      |
|    time_elapsed    | 26987    |
|    total_timesteps | 5103816  |
| train/             |          |
|    actor_loss      | 12.1     |
|    critic_loss     | 0.601    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 70.6     |
|    ep_rew_mean     | 35.5     |
| time/              |          |
|    episodes        | 103600   |
|    fps             | 189      |
|    time_elapsed    | 27521    |
|    total_timesteps | 5204736  |
| train/             |          |
|    actor_loss      | 11.7     |
|    critic_loss     | 0.552    |
|    ent_coef        | 0.0629   |
|    ent_coef_loss   | -0.00648 |
|    learning_rate   | 0.0001   |
|    n_updates       | 433719   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 78.9     |
|    ep_rew_mean     | 51.5     |
| time/              |          |
|    episodes        | 103700   |
|    fps             | 189      |
|    time_elapsed    | 27560    |
|    total_timesteps | 5212212  |
| train/             |          |
|    actor_loss      | 11.9     |
|    critic_loss     | 0.633    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 84       |
|    ep_rew_mean     | 60.6     |
| time/              |          |
|    episodes        | 105100   |
|    fps             | 189      |
|    time_elapsed    | 28158    |
|    total_timesteps | 5325132  |
| train/             |          |
|    actor_loss      | 10.4     |
|    critic_loss     | 0.59     |
|    ent_coef        | 0.0634   |
|    ent_coef_loss   | -0.242   |
|    learning_rate   | 0.0001   |
|    n_updates       | 443752   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81       |
|    ep_rew_mean     | 52       |
| time/              |          |
|    episodes        | 105200   |
|    fps             | 189      |
|    time_elapsed    | 28201    |
|    total_timesteps | 5333256  |
| train/             |          |
|    actor_loss      | 10.3     |
|    critic_loss     | 0.528    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 297      |
|    ep_rew_mean     | 565      |
| time/              |          |
|    episodes        | 106600   |
|    fps             | 189      |
|    time_elapsed    | 29396    |
|    total_timesteps | 5558496  |
| train/             |          |
|    actor_loss      | 7.7      |
|    critic_loss     | 0.614    |
|    ent_coef        | 0.0646   |
|    ent_coef_loss   | -0.159   |
|    learning_rate   | 0.0001   |
|    n_updates       | 463199   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 930      |
|    ep_rew_mean     | 2.23e+03 |
| time/              |          |
|    episodes        | 106700   |
|    fps             | 189      |
|    time_elapsed    | 30034    |
|    total_timesteps | 5678568  |
| train/             |          |
|    actor_loss      | 4.11     |
|    critic_loss     | 0.755    |
|    ent_coef 

In [ ]:
model.save("trained_models/g1-reptile/sac")